<a href="https://colab.research.google.com/github/ViktoriyaMarochkina/Project_data_anallysis/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключим необходимые для работы библиотеки

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Загрузим файл с данными с гугл диска

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/young-people-survey.csv', sep = ";")

Mounted at /content/drive


Выделим из основной таблицы только интересующие нас данные (данные о музыке и фильмах) в две новые таблицы.
Здесь мы используем функцию read_csv() из библиотеки Pandas для загрузки данных из CSV-файла в объект DataFrame. Затем мы выбираем только нужные столбцы, указав их имена в списке selected_columns.


In [4]:
selected_columns = ['Slow songs or fast songs', 'Dance', 'Folk', 'Country', 'Classical music', 'Musical', 'Pop', 'Rock',
                    'Metal or Hardrock', 'Punk', 'Hiphop, Rap',
                    'Reggae, Ska', 'Swing, Jazz', 'Rock n roll', 'Alternative', 'Latino', 'Techno, Trance', 'Opera']
music = data[selected_columns]

selected_columns = ['Horror', 'Thriller', 'Comedy', 'Romantic', 'Sci-fi', 'War', 'Fantasy/Fairy tales', 'Animated', 'Documentary', 'Western']
movies = data[selected_columns]

print(music.shape)
print(music.head())
print("\n")
print(movies.shape)
print(movies.head())


(1010, 18)
   Slow songs or fast songs  Dance  Folk  Country  Classical music  Musical  \
0                       3.0    2.0   1.0      2.0              2.0      1.0   
1                       4.0    2.0   1.0      1.0              1.0      2.0   
2                       5.0    2.0   2.0      3.0              4.0      5.0   
3                       3.0    2.0   1.0      1.0              1.0      1.0   
4                       3.0    4.0   3.0      2.0              4.0      3.0   

   Pop  Rock  Metal or Hardrock  Punk  Hiphop, Rap  Reggae, Ska  Swing, Jazz  \
0  5.0   5.0                1.0   1.0          1.0          1.0          1.0   
1  3.0   5.0                4.0   4.0          1.0          3.0          1.0   
2  3.0   5.0                3.0   4.0          1.0          4.0          3.0   
3  2.0   2.0                1.0   4.0          2.0          2.0          1.0   
4  5.0   3.0                1.0   2.0          5.0          3.0          2.0   

   Rock n roll  Alternative  Lati

Проверим таблицы на наличие пропусков.
Для поиска пропусков (NULL, NaN, None и т.д.) в столбцах таблицы будем использовать функцию isnull() из библиотеки Pandas. Эта функция возвращает таблицу той же формы, что и исходная, но заполненную значениями True и False, указывающими, является ли значение в соответствующей ячейке пропуском.
Нам достаточно определить есть ли хотя бы один пропуск, поэтому будем использовать функцию any(), выведется таблица с булевыми значениями, указывающими, содержит ли каждый столбец хотя бы один пропуск.

In [5]:
has_null = music.isnull().any()
print(has_null)
print("\n")
has_null = movies.isnull().any()
print(has_null)

Slow songs or fast songs    True
Dance                       True
Folk                        True
Country                     True
Classical music             True
Musical                     True
Pop                         True
Rock                        True
Metal or Hardrock           True
Punk                        True
Hiphop, Rap                 True
Reggae, Ska                 True
Swing, Jazz                 True
Rock n roll                 True
Alternative                 True
Latino                      True
Techno, Trance              True
Opera                       True
dtype: bool


Horror                 True
Thriller               True
Comedy                 True
Romantic               True
Sci-fi                 True
War                    True
Fantasy/Fairy tales    True
Animated               True
Documentary            True
Western                True
dtype: bool


Теперь заполним пустые ячейки вычисляя среднеее значение в каждой строке.

Для нашего исследования требовалось разделить жанры музыки и фильмов на два класса. Спокойные и Энергичные/Динамичные. Сделаем это вручную, основываясь на эмпирическом опыте. Используем это разделение для заполнения пустот. Пердположим, что если человек предпочитает более спокойную музыку, то в соответствующих колонках должны стоять более высокие знчения (противоположное аналогично), поэтому при вычислении среднего значения для расчёта будем использовать только те значения, столбцы которых находятся в том же классе, что и пустая ячейка.

МУЗЫКА

Спокойные жанры:
- Classical music, Opera
- Jazz, Swing
- Reggae, Ska
- Country, Musical

Энергичные жанры:
- Dance, Pop, Rock, Metal or Hardrock, Punk, Folk
- Hiphop, Rap
- Rock n roll, Alternative, Latino
- Techno, Trance

ФИЛЬМЫ

Спокойные жанры:
- Romantic
- Fantasy/Fairy tales
- Animated
- Documentary

Динамичные жанры:
- Horror
- Thriller
- Comedy
- Sci-fi
- War
- Western


Для удобства создадим 4 словаря согласно нашему разделению. Метод tolist() возвращает список элементов из объекта Pandas, где первый элемент имеет индекс 0. Поэтому нумеровать будем с 0.


In [20]:
music_list_slow = {'Classical music': 4,
                   'Musical': 5,
                   'Opera': 17,
                   'Swing, Jazz': 12,
                   'Reggae, Ska': 11,
                   'Country': 3}

music_list_fast = { 'Dance': 1,
                    'Folk': 2,
                    'Pop': 6,
                    'Rock': 7,
                    'Metal or Hardrock': 8,
                    'Punk': 9,
                    'Hiphop, Rap': 10,
                    'Rock n roll': 13,
                    'Alternative': 14,
                    'Latino': 15,
                    'Techno, Trance': 16}

movies_list_calm = [3,6,7,8]
movies_list_dynamic = [0,1,2,4,5,9]

Далее мы используем метод iterrows() для обхода строк таблицы и метод isnull() для поиска пропущенных значений в строке. Затем мы находим номера столбцов с пропущенными значениями и обходим их, вычисляя новое значение для каждого столбца в соответствии со словарями music_list_slow и music_list_fast. Используем метод at для изменения значения в таблице.

Таким образом, в результате выполнения этого кода для каждой строки, где значение в столбце column равно NaN, будет вычислено среднее значение в других столбцах из списка music_list_slow. Эти средние значения будут использоваться для замены пропущенных значений в столбце column.

Затем мы используем метод round(), чтобы округлить результат до ближайшего целого значения, и метод astype(int), чтобы преобразовать результат в целое число.

In [14]:
# функция для вычисления максимального значения для каждой строки
def get_max(row):
    # вычисляем среднее значение для столбцов из списка music_list_slow
    mean_values_s = row[list(music_list_slow.keys())].mean()
    # вычисляем среднее значение для столбцов из списка music_list_fast
    mean_values_f = row[list(music_list_fast.keys())].mean()
    # выбираем максимальное значение
    max_value = max(mean_values_s, mean_values_f)
    return max_value

In [17]:
# обход строк таблицы
for index, row in music.iterrows():
    # поиск пропущенных значений в строке
    null_count = row.isnull().sum()
    if null_count > 0:
        # нахождение названий столбцов с пропущенными значениями
        null_columns = row.index[row.isnull()].tolist()

        # обход столбцов с пропущенными значениями
        for null_column in null_columns:
            # вычисление нового значения для пропущенной ячейки

            if null_column in music_list_slow:
                # находим другие столбцы, которые есть в списке music_list_slow
                other_columns = [col for col in music_list_slow if col != null_column]
                # вычисляем среднее значение в других столбцах для данной строки
                new_value = row[other_columns].mean().round().astype(int)
                # замена пропущенного значения на новое значение
                music.at[index, null_column] = new_value
            elif null_column in music_list_fast:
                # находим другие столбцы, которые есть в списке music_list_fast
                other_columns = [col for col in music_list_fast if col != null_column]
                # вычисляем среднее значение в других столбцах для данной строки
                new_value = row[other_columns].mean().round().astype(int)
                # замена пропущенного значения на новое значение
                music.at[index, null_column] = new_value

In [19]:
has_null = music.isnull().any()
print(has_null)

Slow songs or fast songs     True
Dance                       False
Folk                        False
Country                     False
Classical music             False
Musical                     False
Pop                         False
Rock                        False
Metal or Hardrock           False
Punk                        False
Hiphop, Rap                 False
Reggae, Ska                 False
Swing, Jazz                 False
Rock n roll                 False
Alternative                 False
Latino                      False
Techno, Trance              False
Opera                       False
dtype: bool
